In [36]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from transformers import BertModel, DistilBertModel
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
import numpy as np
import os.path
from datasets import load_from_disk
import pickle as pkl
from sklearn.linear_model import LogisticRegressionCV
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm
import dvu
dvu.set_style()
import sys
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from os.path import join as oj
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import imodels
pd.set_option('display.max_rows', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
dset = datasets.load_dataset('sst2')['train']
dset = dset.select(np.random.choice(len(dset), size=500, replace=False))
v = CountVectorizer()
X = v.fit_transform(dset['sentence'])
y = dset['label']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
X_train = X_train.todense()
X_test = X_test.todense()
feature_names = v.get_feature_names_out().tolist()

/home/chansingh/.embgam/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Using custom data configuration default
Found cached dataset sst2 (/home/chansingh/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
params_dt = {'max_leaf_nodes': [5]}
dt_cv = GridSearchCV(DecisionTreeClassifier(), params_dt)
params_rf = {'max_rules': [5]}
rf_cv = imodels.RuleFitClassifier(**params_rf)
clfs = {
    'dt': dt_cv,
    # 'logistic': LogisticRegressionCV(),
    # 'figs': imodels.FIGSClassifierCV(),
    'rulefit': rf_cv,
}
r = defaultdict(list)

for k in clfs.keys():
    clf = clfs[k]
    try:
        clf.fit(X_train, y_train, feature_names=feature_names)
    except ValueError as e:
        clf.fit(X_train, y_train)
    r[k].append(clf.score(X_test, y_test))

In [48]:
pd.DataFrame.from_dict(r)

,dt,rulefit
0,0.545455,0.466667


In [49]:
dt = clfs['dt'].best_estimator_
print(export_text(dt, feature_names=feature_names))

|--- much <= 0.50
|   |--- than <= 0.50
|   |   |--- hollywood <= 0.50
|   |   |   |--- obvious <= 0.50
|   |   |   |   |--- class: 1
|   |   |   |--- obvious >  0.50
|   |   |   |   |--- class: 0
|   |   |--- hollywood >  0.50
|   |   |   |--- class: 0
|   |--- than >  0.50
|   |   |--- class: 0
|--- much >  0.50
|   |--- class: 0



In [50]:
# clfs['rulefit'].best_estimator_.feature_names = feature_names
print(clfs['rulefit'])

> ------------------------------
> RuleFit:
> 	Predictions are made by summing the coefficients of each rule
> ------------------------------
                                                                                                                                                         rule  coef
                                                                                                                                                   you <= 0.5 -0.16
no <= 0.5 and one <= 0.5 and science <= 0.5 and script <= 0.5 and story <= 0.5 and than <= 0.5 and too <= 0.5 and fluke <= 0.5 and it <= 0.5 and lives <= 0.5  0.39
                                                                                        much <= 0.5 and or <= 0.5 and be <= 0.5 and is <= 0.5 and long <= 0.5  0.13



# Try LLM version